In [56]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [57]:
#Read CSV
merge_output = pd.read_csv("./zillow_census_m_OUTPUT.csv")


In [58]:
#Fill NaN with 0 (NaN values are for international migration. We are only interested in domestic migration)
merge_output = merge_output.fillna(0)

In [59]:
#Split County into two County and State columns
county_split = merge_output["County"].str.split(pat=', ', expand=True)
merge_output["State"] = county_split[1]
merge_output['State'].nunique()
merge_output.head()

,Unnamed: 0,County,2009IN,2009NET,state,county,2010IN,2010NET,2011IN,2011NET,...,2015 Difference,2016 Difference,2010 Percent Change,2011 Percent Change,2012 Percent Change,2013 Percent Change,2014 Percent Change,2015 Percent Change,2016 Percent Change,State
0,0,"Acadia County, Louisiana",2082.0,-862.0,660,30,2129.0,-45.0,1996.0,-572.0,...,2800,2100.0,7.008086,-6.675063,0.269906,9.555855,3.439803,3.325416,2.413793,Louisiana
1,1,"Ada County, Idaho",26673.0,4034.0,3136,196,25963.0,3388.0,22795.0,1299.0,...,14500,24400.0,-8.181316,-7.043949,6.282383,15.661182,4.109589,7.338057,11.504008,Idaho
2,2,"Adair County, Missouri",2389.0,643.0,1537,53,2245.0,547.0,2602.0,826.0,...,5400,5400.0,-11.207970,-4.207574,-1.464129,0.000000,5.943536,7.573633,7.040417,Missouri
3,3,"Adams County, Colorado",36313.0,5386.0,1336,167,36404.0,4927.0,36864.0,3041.0,...,35500,31700.0,-3.395785,-0.606061,8.231707,9.746479,13.911704,15.998197,12.315462,Colorado
4,4,"Adams County, Idaho",190.0,-97.0,112,21,192.0,-8.0,237.0,123.0,...,11600,38300.0,-10.820669,-3.340150,2.538787,8.940853,5.429293,6.946108,21.444569,Idaho


In [60]:
#Get total NetValues
net_df = merge_output.loc[:, ["State", "County","2009NET", "2010NET", "2011NET", "2012NET", "2013NET", "2014NET", "2015NET"]]
difference_df = merge_output.loc[:,["2010 Percent Change", "2011 Percent Change", "2012 Percent Change", "2013 Percent Change", "2014 Percent Change", "2015 Percent Change", "2016 Percent Change"]]


In [61]:
#Groupby State
#state_migration = merge_output_net.groupby(["State"], as_index=False).sum()
#state_migration

In [62]:
#Create column that has net total migration
net_df["Total Net"] = state_migration.sum(axis=1)


In [68]:
years = ['2010', '2011', '2012', '2013', '2014', '2015']
for year in years:
    plt.figure(dpi=1200)
    
    merge_output.plot(kind="scatter", x=merge_output[f"{year}NET"], y=merge_output[f"{year} Difference"], grid=True, figsize=(20,10),
              title=f"Change In Median Housing Price Vs. Net Migration {year}")

    plt.ylabel("Zillow Median List Price")
    plt.xlabel("Net Migration")
   # plt.savefig(f"netMigration{year}.png",dpi = 1000)
    #fig, (ax1) = plt.subplots(1, sharex=True)
    #fig.suptitle("Regression", fontsize=16, fontweight="bold")

    #ax1.set_xlim(min(difference), max(difference))
    #ax1.set_ylim(-20000,80000)
    #ax1.plot(difference, moved_net, linewidth=0, marker="o")
    #ax1.plot(difference, vc_fit, "b--", linewidth=1)
    #ax1.set_ylabel("Moved Net")

KeyError: '[ -45. 3388.  547. ... 2257. 1363. 2597.] not in index'

<Figure size 7200x4800 with 0 Axes>